In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm
import seaborn as sns    

sys.path.append(os.path.abspath("../"))
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [3]:
import importlib

import modules
import fe_modules

importlib.reload(modules)
importlib.reload(fe_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from fe_modules.text_manipulation import get_domain
from fe_modules.datetime_features import get_timestamp, get_relative_time, part_of_day_to_hour, add_hour_to_date
from fe_modules.preprocessing import clean_os_type

# Load data

In [4]:
LOCAL_DATA_PATH = '../data/'
SPLIT_SEED = 42

In [16]:
df.loc[860]

url                                         http://gamesisart.ru
title                     GamesisArt.ru — Главная страница сайта
uri                             https://gamesisart.ru/index.html
description    Сайт о компьютерных играх с точки зрения искус...
site_name                        Компьютерные игры как искусство
keywords       Компьютерные,Видео,игры,Искусство,Шедевры,лучш...
text           GamesisArt.ru — Главная страница сайтаПерейти ...
Name: 860, dtype: object

In [32]:
df

,url,title,uri,description,site_name,keywords,text,tod
12,http://zen.yandex.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,http://banners.adfox.ru,Яндекс,NaN,Найдётся всё,NaN,NaN,"Ой!Подтвердите, что запросы отправляли вы, а н...",NaN
52,http://frontend.vh.yandex.ru,Яндекс,NaN,Найдётся всё,NaN,NaN,"Ой!Подтвердите, что запросы отправляли вы, а н...",NaN
124,http://news.yandex.ru,Яндекс,NaN,Найдётся всё,NaN,NaN,"Ой!Подтвердите, что запросы отправляли вы, а н...",NaN
132,http://static.kost.tv,Kost.TV,NaN,NaN,NaN,NaN,Kost.TVПочему мы?ФорматыРекламодателямВебмасте...,NaN
...,...,...,...,...,...,...,...,...
9609,http://mundolatino.org.ua,"Индийские и турецкие сериалы 2019 год, Латино...",NaN,NaN,NaN,"Турецкие сериалы 2019, мексиканские сериалы, и...","Индийские и турецкие сериалы 2019 год, Латино...",NaN
9651,http://pefl.ru,PEFL - футбольный менеджер онлайн,NaN,NaN,NaN,NaN,PEFL - футбольный менеджер онлайнВход:Регистра...,NaN
9689,http://lektsia-com.turbopages.org,Лекция — Главная,NaN,NaN,NaN,"лекция, конспект, реферат, доклад, курсовая, р...",﻿Лекция — ГлавнаяАрхитектураАудитВоенная наука...,NaN
9764,http://winx-fan.ru,"WINX-FAN - ИГРЫ, МУЛЬТИКИ, РАСКРАСКИ, ФОТО ВИНКС",NaN,NaN,NaN,"Винкс, Игры Винкс, мультфильмы Винкс смотреть,...","WINX-FAN - ИГРЫ, МУЛЬТИКИ, РАСКРАСКИ, ФОТО ВИН...",NaN


In [4]:
df=pd.read_excel('../executable/sites_new.xls',index_col=0)

In [5]:
dfm=pd.read_excel('../external_data/sites_final.xlsx',index_col=0)

In [6]:
def text_or_desc(row):
    if pd.isnull(row['description']):
        return row['text']
    return row['description']

df['tod']=df[['description','text']].apply(lambda x: text_or_desc(x),axis=1)

In [7]:
df= df[['url','tod']]

In [8]:
df

,url,tod
12,http://zen.yandex.ru,NaN
46,http://banners.adfox.ru,Найдётся всё
52,http://frontend.vh.yandex.ru,Найдётся всё
124,http://news.yandex.ru,Найдётся всё
132,http://static.kost.tv,Kost.TVПочему мы?ФорматыРекламодателямВебмасте...
...,...,...
9609,http://mundolatino.org.ua,"Индийские и турецкие сериалы 2019 год, Латино..."
9651,http://pefl.ru,PEFL - футбольный менеджер онлайнВход:Регистра...
9689,http://lektsia-com.turbopages.org,﻿Лекция — ГлавнаяАрхитектураАудитВоенная наука...
9764,http://winx-fan.ru,"WINX-FAN - ИГРЫ, МУЛЬТИКИ, РАСКРАСКИ, ФОТО ВИН..."


In [9]:
df = dfm.merge(df.rename(columns={"url": "url_host", 
                                        "tod":'missing_tod'}), on="url_host", how="left")

In [10]:
df.loc[dfm.tod.str.contains('Ë').fillna(False) | dfm.tod.str.contains('Ð').fillna(False)].tod = df.missing_tod

In [12]:
del df['missing_tod']

In [13]:
df.loc

,url_host,vpn_only,tod,is_broken
0,http://googleads.g.doubleclick.net,1,Google Marketing Platform offers an enterprise...,0
1,http://yandex.ru,0,Найдётся всё,0
2,http://i.ytimg.com,0,����_x0000__x0010_JFIF_x0000__x0001__x0001__x0...,0
3,http://vk.com,0,ВКонтакте – универсальное средство для общения...,0
4,http://avatars.mds.yandex.net,0,410 Gone410 Gonenginx,1
...,...,...,...,...
9995,http://tolstoy-lit.ru,1,Ëåâ ÒîëñòîéËåâ Íèêîëàåâè÷ ÒîëñòîéÑî÷èíåíèÿ:Àíí...,0
9996,http://neprizyvnoi.ru,0,Все об армии и призыве: информационная помощь ...,0
9997,http://wik-end.com,0,Информационный портал Weekend-Тверь - всегда с...,0
9998,http://probeg.org,1,ПроБЕГ в России и миреКалендариПолныйМосква и ...,0
